In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
import geopandas as gpd 
import snappy
import earthpy.plot as ep
import rasterio
from rasterio.plot import reshape_as_raster, reshape_as_image

%matplotlib inline

# Change module setting
pd.options.display.max.colwidth = 80

# User input

# Path to be processed and co-located Sentinel-1 and Sentinel-2 stack
S1_S2_stack = '/shared/Training/PY02_CropMapping/Processing/S1_S2_stack.tif'

# Path to training and Validation point shapefiles
training_points = '/shared/Training/PY02_CropMapping/AuxData/Shapefiles/Training_points1000.shp'
validation_points = '/shared/Training/PY02_CropMapping/AuxData/Shapefiles/Validation_points200.shp'

class_names = ['Maize', 'Soybean', 'Winter Crop - Soybean', Winter crop - Maize', 'Peanut', 'Rangeland/Pastures', 'Natural woody vegetation', 'Masked'] 

#Eight classes represented in our training data.

# Create a custom color map 

# create a color dictionary with specific colors assigned to specific crop type.

colors = dict((
	(0, (0,76,153,255)), #Blue - Maize
	(1, (0,153,0,255)), #Green - Soybean
	(2, (255,0,0,255)), #Red Winter Crop - Soybean
	(3, (255,153,51,255)), #Orange Winter Crop - Maize
	(4, (255,255,0,255)), #Yellow - Peanut
	(5, (204,153,255,255)), #Purple - Rangeland/Pasture
	(6, (178,255,102,255)), #Light green - Natural woody vegetation
	(7, (0,0,0,255)), #Black - Masked
	
))

#Convert 0-255 values to float in rage 0-1

for k in colors:
	v = colors[k]
	_v = [ v / 255.0 for _v in v]
	colors[k] = _v
	
# Create matplotlib_colormap from a list of colors and call it Classification
index_colors = [colors[key] if key in colors else (1, 1, 1, 0) for key in range(0,8)]
cmap = plt.matplotlib.colors.ListedColormap(index_colors, 'Classification', 8)

# 3. Load and visualize input data
# 3.1 visualize a RGB composite of first Sentinel-2 image and first Sentinel-1 image

#Load the input dataset into a dataset object
src = rasterio.open(S1_S2_stack)

#Get list of band names, uses  snappy package
prep_vis = snappy.ProductIO.readProduct(S1_S2_stack)
bands = list(prep_vis.getBandNames())

# read all bands of the dataset into 3-dimensional array stack - (band, row, column)
stack = src.red()
print(stack.shape)
# Create a figure and plot the RGB composites of first Sentinel-2 image (True color) and first Sentinel-1 image (Sual Pol Ration VV+VH)
# The band numbers must be changed accordingly to you input dataset

fig, (ax1, ax2) = plt.supblot(1, 2, figsize=(20, 10))
ax1 = ep.plot_rgb(arr = stack, rgb = (2, 1, 0), stretch=True, ax = ax1, title="RGB Composite - Sentinel-2")
stack_s1 = np.stack((stack[60], stack[61]; stack[61]/stack[60]))
ax2 = ep.plot_rgb(arr = stack_s1, rgb = (1, 0, 2), stretch=True, ax = ax2, title="RGB Composite - Sentinel-1( VV, VH, VV/VH)")
plt.tight_layout()

# 4. Create Training/Validation data
# create an exact copy of your input dataset and saves it in memory file.

# Load our original input file bands to a numpy array stack
img = src.read()
# The Copy the profile of the original GeoTIFF input file
profile = src profile
# The copy the profile of the original GeoTIFF input file
with rasterio.io.MemoryFile() as memfile:
	with memfile.open(**profile) as dst:
		for i in range(0,src.count):
			dst.write(img[i], i+1)
	dataset = memfile.open()
	
	
# 4.2 Extract training dataset 

# Read points from shapefile

train_pts = gpd.read_file(training_points)
# These are the attributes in out dataset
train_pts = train_pts[['GRIDCODE', 'UTM_E', 'UTM_N', 'geometry']]
train_pts.index = range(len(train_pts))
# Create list of point coordinates
coords = [(x,y)] for x, y in zip(train_pts.UTM_E, train_pts.UTM_N)]

# Sample the each band of raster dataset at each point in the coordinate list
train_pts['Raster Value'] = [x for x in dataset.sample(coords)]
# All bands value are saved as a list in the Raster Value Column 
# Unpack the Raster Value column to separate column for each band - band names were retrieved with snappy and are now usef as column names
train_pts[bands] = pd.DataFrame(train_pts['Raster Value'].tolist(), index= train_pts.index)
train_pts = train_pts.drop(['Raster Value'], axis=1) # remove Raster Value column
# Change the values for last three classes - original class number [0,1,2,3,4,7,8,15] -> [0,1,2,3,4,5,6,7]
train_pts['GRIDCODE'] = train_pts['GRIDCODE'].replace([7,8,15],[5,6,7])
train_pts.to_csv('train_pts.csv') # Save our training dataset to CSV 
train_pts.head() # Visualize the first rows of the dataframe 
# Check if class numbers were correctly changed 
train_pts['GRIDCODE'].unique()

# Plot Class profiles over our dataset (September 2018 - June 2019)

prof = train_pts.groupby(['GRIDCODE']).mean() # calculate average value for each class in each band 
fig = plt.figure(figsize = (17,20))
band_n = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'VH', 'VV'] # bands we included in the analysis for each name we have multiple dates included
n =1 # counter

for ba in band_n: # Iterate over band names
	ax = fig.added_subplot(4,2,n)
	ax.title.set_text(ba)
	band_val = prof[prof.columns[prof.columns.to_series().str.contains(ba)]] # Select all columns in the dataframe containing a band name e.g B2
	for index, row in band_val.iterrows(): # Plot line for each class in the selected band
		color = cmap(index)
		ax.plot(row, color=color)
	ax.set_xticklables([str(x) for x in range(1, len(row)+1)]) # Replace column names with numbers
	ax.legends(loc="best", fontsize="small", ncol=2, labels=class_names)
	n = n+1
	
# Split training dataset to lables (y) and input features (x)
y = train_pts['GRIDCODE'].values
# We create a different x dataset based on the subset we want to use for training our model
x = []
x.append(train_pts[[b for b in bands if "Sigma0" in b]].values) # Only Sentinel-1 data
x.append(train_pts[[b for b in bands if "B" in b]].values) # Only Sentinel-2 data
x.append(train_pts[bands].values) # all Sentinel-1 and Sentinel-2 data
del(train_pts, coords) 

# Check the shapes of our features datasets
print('The training data sizes are: Sentinel-1 stack:', x[0].shape, 'Sentinel-2 stack:', x[1].shape,'Sentinel-1 and Sentinel-2 stack: ', x[2].shape)

# 4.3 Extract validation dataset 

valid_pts = gpd.read_file(validation_points)
valid_pts = valid_pts[['GRIDCODE', 'UTM_E', 'UTM_N', 'geometry']] # These are the attributes in our point dataset
valid_pts.index = range(len(valid_pts))
coords = [(x,y) for x,y in zip(valid_pts.UTM_E, valid_pts.UTM_N)] # Create list of point coordinates
# Sample the each band of raster dataset at each point in the coordinate list
valid_pts['Raster Value'] = [x for x in dataset.sample(coords)] # All band values are saved as a list in the Raster Value column
# Unpack the Raster Value colun to separate column for each band - band names were retrieved with snappy and are now usef as column
# names

valid_pts[bands] = pd.DataFrame(valid_pts['Raster Value'].tolist(), index= valid_pts.index)
valid_pts = valid_pts.drop(['Raster Value'], axis=1) # Remove Raster Value column
# Change the values for last three classes - original class numbers [0,1,2,3,4,7,8,15] -> [0,1,2,3,4,5,6,7]
valid_pts['GRIDCODE'] = valid_pts['GRIDCODE'].replace([7,8,15],[5,6,7])
valid_pts.to_csv('valid_pts.csv') # Save our validation dataset to CSV
valid_pts.head() # visualize the first rows of the dataframe

# Split validation dataset to training and validation data 
y_valid = valid_pts['GRIDCODE'].values
# We create a different x dataset based on the subset we want to use for training our model
x_valid = []
x_valid.append(valid_pts[b for b in bands if "Sigma0" in b]].values) # Only Sentinel-1 data
x_valid.append(valid_pts[b for b in bands if "B" in b]].values) # Only Sentinel-2 data
x_valid.append(valid_pts[bands].values) # All Sentinel-1 and Sentinel-2 data 
del(valid_pts, coords)

# Check the shapes of our features datasets
print('The validation data sizes are: Sentinel-1 stack: ', x_valid[0].shape, 'Sentinel-2 stack', x_valid[1].shape, 'Sentinel-1 and Sentinel-2 stack: ', x_valid[2].shape

# Train the RF classifier 

class sklearn.ensemble.RandomForestClassification(n_estimators = 100, *, criterion ='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_node=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

from sklearn.ensemble import RandomForestClassifier

model_RF=[]
for i in range(0,3): # Loop over our three features dataset and fir a RF model to each
	rf = RandomForestClassifier(n_estimators=300, oob_score=True, max_features='auto') # Initialize our model with 500 trees and the default  
	rf = rf.fit(x[i], y) # Fit the model to the training dataset
	model_RF.append(rf) # Save the trained model to a list
rf = None
# The OOB score of the training dataset obtained using an out-of-bag estimate
print('Our OOB prediction of accuracy for S1 stack is: {oob}%'.format(oob=model_RF[0].oob_score *100))
print('Our OOB prediction of accuracy for S2 stack is: {oob}%'.format(oob=model_RF[1].oob_score *100))
print('Our OOB prediction of accuracy for S1 and S2 stack is: {oob}%'.format(oob=model_RF[2].oob_score *100))

# 5.2 RF Model Validation

from sklearn.metrics import classification_report

label = ['Sentinel-1 only', 'Sentinel-2 only', 'Sentinel-1 and Sentinel-2 stack']
# Run predictions on the test dataset 
for i in range(0,3): # Loop over the trained RF models
	rf = model_RF[i] # Load the trained model
	y_pred = rf.predict(x_valid[i])
	print(lable[i])
	print(classification_report(y_valid, y_pred, target_names=class_names)) # Print the classification report 
	
# 5.3 Classify the iage with the trained RF model

# Take our full image and reshape into long 2d array (nrow * ncol, nband) for classification 
# You may need to reduce image size if your kernel crashes, as this step takes a lot of memory 
imf = dataset.read() # img = dataset.read()[:,150:600, 250:1400]
print(img.shape) # (bands, rows, cols)
reshaped_img = reshape_as_image(img)
print(reshaped_img.shape) # (row, cols, bands)
# Reshape to 2D array 
band_num_S1 = [bands.index(l) for l in bands if "Sigma0" in l][0] # start index for Sentinel-1 bands
band_num_S2 = [bands.index(l) for l in bands if "B" in l][0]      # start index for Sentinel-2 bands
# Create a reshaped image input for each of our trained models (used also in SVM)
class_input_S1 = reshaped_img[:,:, band_num_S1: band_num_S1 + x[0].shape[1]].reshape(-1, x[0].shape[1])
class_input_S2 = reshaped_img[:,:, band_num_S2: band_num_S2 + x[1].shape[1]].reshape(-1, x[1].shape[1])
class_input_S1S2 = reshaped_img.reshape(-1, x[2].shape[1])
print(class_input_S1S2.shape) # (rows*cols, bands)

# Sentinel-1 stack classification 
rf_S1 = model_RF[0]
class_RF_S1 = rf_S1.predict(class_input_S1)
# Reshape our classification map back into a 2D matrix so we can visualize it 
class_RF_S1 = class_RF_S1.reshape(reshaped_img[:, :, 0].shape)

# Sentinel-2 stack classification 
rf_S2 = model_RF[1]
class_RF_S2 = rf_S2.predict(class_input_S2)
# Reshape our classification map back into a 2D matrix so we can visualize it 
class_RF_S2 = class_RF_S2.reshape(reshaped_img[:, :, 0].shape)

# Sentinel-1 and Sentinel-2 stack classification 
rf_S1S2 = model_RF[2]
class_RF_S1S2 = rf_S1S2.predict(class_input_S1S2)
# Reshape our classification map back into a 2D matrix so we can visualize it 
class_RF_S1S2 = class_RF_S1S2.reshape(reshaped_img[:, :, 0].shape)

# 5.4 Visualize RF results

fig, ((ax1, ax2, ax3)) = plt.subplot(1,3, figsize=(21, 21))
ax1.set_title("Random Forest - Sentinel-1 Classification')
im1 = ax1.imshow(class_RF_S1, cmap=cmap, interpolation='none')
ax2.set_title("Random Forest - Sentinel-2 Classification')
im2 = ax2.imshow(class_RF_S2, cmap=cmap, interpolation='none')
ax3.set_title("Random Forest - Sentinel-1 and Sentinel-2 Classification')
im3 = ax3.imshow(class_RF_S1S2, cmap=cmap, interpolation='none')

# Create a legend with class names and colors 
patches = [mtp.patches.Patch(color=cmap(i), label="{:s}".format(class_names[i])) for i in range(len(class_names)) ]
fig.legend(handles=patches, loc='center', bbox_to_anchor=(0.5, 0.37), ncol=9)

# Save our classification results into a GeoTIFF 
with rasterio.Env():
	# Write an array as a raster band to a new 8-bit file. For 
	# the new file's profile, we start with the profile of the source
	profile = src.profile
	# Change the band count to 3, set the dtype to uint8, and specify LZW compression.
	with rasterio.open('RF_classification.tif', 'w', **profile) as dst:
		dst.write(class_RF_S1.astype(restio.unit8), 1)
		dst.write(class_RF_S2.astype(restio.unit8), 2)
		dst.write(class_RF_S1S2.astype(restio.unit8), 3)
# At the end of the 'with rasterio.Env()'' block, context manager exists and all drivers are de-registered.
